In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
dataset, info = tfds.load("tf_flowers", as_supervised=True,
                         with_info=True )

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [3]:
info

tfds.core.DatasetInfo(
    name='tf_flowers',
    full_name='tf_flowers/3.0.1',
    description="""
    A large set of images of flowers
    """,
    homepage='https://www.tensorflow.org/tutorials/load_data/images',
    data_dir='/root/tensorflow_datasets/tf_flowers/incomplete.7K037P_3.0.1/',
    file_format=tfrecord,
    download_size=218.21 MiB,
    dataset_size=221.83 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=5),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=3670, num_shards=2>,
    },
    citation="""@ONLINE {tfflowers,
    author = "The TensorFlow Team",
    title = "Flowers",
    month = "jan",
    year = "2019",
    url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }""",
)

In [4]:
dataset_size = info.splits["train"].num_examples
dataset_size

3670

In [5]:
class_names = info.features["label"].names
class_names

['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

In [6]:
n_classes = info.features["label"].num_classes
n_classes

5

In [7]:
test_set_raw, valid_set_raw, train_set_raw=tfds.load(
    "tf_flowers",
    split=["train[:10%]","train[10%:25%]","train[25%:]"],
    as_supervised=True
)

In [8]:
batch_size = 32
preprocess=tf.keras.Sequential([
    tf.keras.layers.Resizing(height=224, width=224,
                             crop_to_aspect_ratio=True),
    tf.keras.layers.Lambda(
        tf.keras.applications.xception.preprocess_input
    )
])

In [9]:
train_set=train_set_raw.map(lambda X,y:(preprocess(X),y))
train_set=train_set.shuffle(1000,seed=42).batch(batch_size).prefetch(1)

valid_set=valid_set_raw.map(lambda X,y:(preprocess(X),y)).batch(batch_size)
test_set=test_set_raw.map(lambda X,y:(preprocess(X),y)).batch(batch_size)


In [10]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal",seed=42), #döndürme
    tf.keras.layers.RandomContrast(factor=0.2, seed=42), #contrast degisme
    tf.keras.layers.RandomRotation(factor=0.05, seed=42) #rastgele döndürme
])

In [11]:
tf.random.set_seed(42)

base_model=tf.keras.applications.xception.Xception(
    weights="imagenet",
    include_top=False, # en üst katmanlar alınmadı
)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [12]:
#parametre sayısı düşürme
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

#outputda kategori kadar nöron, softmax->çok kategorili problemler
output=tf.keras.layers.Dense(n_classes, activation="softmax")(x)

model = tf.keras.Model(inputs=base_model.input,
                       outputs=output)

In [13]:
#önceden egitilmiş katmanları dondur,up eğitim dışı tutma
for layer in base_model.layers:
  layer.trainable = False

In [14]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.SGD(
        learning_rate=0.1,
        momentum=0.9
    ),
    metrics=["accuracy"]
)

In [15]:
history=model.fit(train_set,
                  validation_data=valid_set,
                  epochs=3)

Epoch 1/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 40s 253ms/step - accuracy: 0.7203 - loss: 0.8681 - val_accuracy: 0.8439 - val_loss: 0.6055
Epoch 2/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 148ms/step - accuracy: 0.8995 - loss: 0.3919 - val_accuracy: 0.8403 - val_loss: 0.7636
Epoch 3/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 21s 155ms/step - accuracy: 0.9444 - loss: 0.1873 - val_accuracy: 0.8766 - val_loss: 0.5510


In [21]:

for indices in zip(range(33), range(33, 66), range(66, 99), range(99, 132)):
    for idx in indices:
        print(f"{idx:3}: {base_model.layers[idx].name:22}", end="")
    print()

  0: input_layer_1          33: block4_pool            66: block8_sepconv1_act    99: block11_sepconv2_act  
  1: block1_conv1           34: batch_normalization_2  67: block8_sepconv1       100: block11_sepconv2      
  2: block1_conv1_bn        35: add_2                  68: block8_sepconv1_bn    101: block11_sepconv2_bn   
  3: block1_conv1_act       36: block5_sepconv1_act    69: block8_sepconv2_act   102: block11_sepconv3_act  
  4: block1_conv2           37: block5_sepconv1        70: block8_sepconv2       103: block11_sepconv3      
  5: block1_conv2_bn        38: block5_sepconv1_bn     71: block8_sepconv2_bn    104: block11_sepconv3_bn   
  6: block1_conv2_act       39: block5_sepconv2_act    72: block8_sepconv3_act   105: add_9                 
  7: block2_sepconv1        40: block5_sepconv2        73: block8_sepconv3       106: block12_sepconv1_act  
  8: block2_sepconv1_bn     41: block5_sepconv2_bn     74: block8_sepconv3_bn    107: block12_sepconv1      
  9: block2_sepconv

In [23]:
#56dan sonrasını eğit
for layer in base_model.layers[56:]:
  layer.trainable = True


In [24]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.SGD(
        learning_rate=0.1,
        momentum=0.9
    ),
    metrics=["accuracy"]
)

In [25]:
history=model.fit(train_set,
                  validation_data=valid_set,
                  epochs=10)


Epoch 1/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 47s 352ms/step - accuracy: 0.9714 - loss: 0.0834 - val_accuracy: 0.8240 - val_loss: 0.9072
Epoch 2/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 26s 292ms/step - accuracy: 0.9679 - loss: 0.1089 - val_accuracy: 0.8929 - val_loss: 0.3200
Epoch 3/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 41s 294ms/step - accuracy: 0.9805 - loss: 0.0512 - val_accuracy: 0.9310 - val_loss: 0.2986
Epoch 4/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 27s 306ms/step - accuracy: 0.9833 - loss: 0.0627 - val_accuracy: 0.8766 - val_loss: 0.4264
Epoch 5/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 26s 287ms/step - accuracy: 0.9935 - loss: 0.0217 - val_accuracy: 0.9074 - val_loss: 0.3820
Epoch 6/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 41s 294ms/step - accuracy: 0.9978 - loss: 0.0090 - val_accuracy: 0.8984 - val_loss: 0.4554
Epoch 7/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 27s 308ms/step - accuracy: 0.9943 - loss: 0.0155 - val_accuracy: 0.9147 - val_loss: 0.4087
Epoch 8/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 39s 287ms/step - accuracy: 0.9941 - loss: 0.0148 - val_accu